### "Simplification" attemps with OSMnx

For test (continents) cities [FUA ID]:
* (Africa) Douala [809]
* (Oceania) Auckland [869]
* (Asia) Aleppo [1133] -- Wuhan [8989]
* (Europe) Liège [1656]
* (South America) Bucaramanga [4617]
* (North America) Salt Lake City [4881]

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-12-05T11:50:41.774192+01:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.30.0

Compiler    : Clang 17.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import os

import geopandas as gpd
import momepy
import networkx as nx
import osmnx as ox

from core import utils, viz

%watermark -w
%watermark -iv

Watermark: 2.5.0

core     : 0.1.dev170+gc3e76d8.d20241205
momepy   : 0.9.1
geopandas: 1.0.1
osmnx    : 2.0.0
networkx : 3.4.2



**Read in meta data**

In [3]:
# read in sample metadata
sample = utils.read_sample_data()
sample.head(2)

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry,continent,iso_a3
305,9129.0,1.0,8078,Gonda,1.0,IND,India,66.0,29.0,1.074100e+06,1.066419e+06,7680.678101,"POLYGON ((81.98398 27.19657, 81.99471 27.19657...",Asia,IND
91,7578.0,6.0,10577;10581;10583;10596;10605;10607,Chongqing,1.0,CHN,China,2267.0,618.0,6.036834e+06,5.157726e+06,879107.861057,"POLYGON ((106.23972 29.52328, 106.19622 29.523...",Asia,CHN


**Read in data for example city**

In [4]:
# loop through FUAs

for fua in utils.fua_city:
    city = utils.fua_city[fua]

    # read in original data
    gdf = utils.read_original(fua)

    # convert to networkX graph
    G = momepy.gdf_to_nx(
        gdf_network=gdf,
        approach="primal",
        directed=True,
        integer_labels=True,
        length="length",
    )
    print(utils.graph_size(f"Momepy-NetworkX Primal Graph: {city}", G))

    # simplify in OSMnx (ie removing interstitial nodes)
    G_simp = ox.simplify_graph(G)
    print(
        utils.graph_size(f"Momepy-NetworkX Primal Graph (simplified): {city}", G_simp)
    )

    # consolidate with default value 10m
    tol = 10
    G_cons = ox.consolidate_intersections(
        G=G_simp,
        tolerance=tol,
        rebuild_graph=True,
        dead_ends=True,
        reconnect_edges=True,
    )
    print(utils.graph_size(f"Consolidated graph with tolerance {tol}m", G_cons))

    # convert into digraph (drop multiedges, choosing by lenght)
    G_cons_di = ox.convert.to_digraph(G_cons, weight="length")

    # make gdf of edge geoms (can't use ox.gdf method on non-multi digraph)
    geoms = list(nx.get_edge_attributes(G_cons_di, "geometry").values())
    edges_cons = gpd.GeoDataFrame(geometry=geoms, crs=G_cons_di.graph["crs"])

    # save result to subfolder
    os.makedirs(f"../../data/{fua}/osmnx/", exist_ok=True)
    edges_cons.to_parquet(f"../../data/{fua}/osmnx/{fua}.parquet")

    print(f"OSMnx simplification done for {city}")

Momepy-NetworkX Primal Graph: Aleppo
	* MultiDiGraph with 64445 nodes and 78908 edges
Momepy-NetworkX Primal Graph (simplified): Aleppo
	* MultiDiGraph with 35480 nodes and 49943 edges
Consolidated graph with tolerance 10m
	* MultiDiGraph with 29333 nodes and 43609 edges
OSMnx simplification done for Aleppo
Momepy-NetworkX Primal Graph: Auckland
	* MultiDiGraph with 57031 nodes and 60364 edges
Momepy-NetworkX Primal Graph (simplified): Auckland
	* MultiDiGraph with 20436 nodes and 23765 edges
Consolidated graph with tolerance 10m
	* MultiDiGraph with 12911 nodes and 15869 edges
OSMnx simplification done for Auckland
Momepy-NetworkX Primal Graph: Bucaramanga
	* MultiDiGraph with 74716 nodes and 79317 edges
Momepy-NetworkX Primal Graph (simplified): Bucaramanga
	* MultiDiGraph with 27983 nodes and 32584 edges
Consolidated graph with tolerance 10m
	* MultiDiGraph with 19761 nodes and 24157 edges
OSMnx simplification done for Bucaramanga
Momepy-NetworkX Primal Graph: Douala
	* MultiDiGraph